In [1]:
# data manipulation
import pandas as pd
import numpy as np

# model training
from sklearn.model_selection import GridSearchCV

# classifiers
from sklearn.naive_bayes import GaussianNB # naive bayes
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.svm import SVC # SVM
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# machine learning models/techniques third parties 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.linear_model as sklm

# for final validation to plot ROC curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

from tensorflow import set_random_seed
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Activation, MaxPool2D, Dropout, Flatten
from keras import optimizers, losses
from keras.models import model_from_json


# to time experiments
import time


# to calculate kurtosis and skew
from scipy.stats import kurtosis
from scipy.stats import skew

# for plots
import seaborn as sns 
import matplotlib as pl
import matplotlib.pyplot as plt

import dataexp as dtex # our code
import dataio as dtio # our code 
import preprocessing as pre # our code

# to display pandas and other elements in html
from IPython.display import display
from IPython.core.display import HTML
from IPython.display import Image

# for ordered dictionaries 
from collections import OrderedDict

# for feature extraction
import feature_extraction as fe # our code

# machine learning models/techniques
import model_validation as me # our code
#from decision_tree import DecisionTree # our code
#from random_forest import RandomForest # our code
from logistic_regression import LogisticRegression # our code





# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
x_train_o = pd.read_csv('financial_data.csv')
y_train_o = pd.read_csv('revealed_businesses.csv')

x_test_o = pd.read_csv("testing_data.csv")

In [3]:
x_train_o.replace('?', np.nan, inplace=True)
x_train_o = x_train_o.astype('float64')


x_test_o.replace('?', np.nan, inplace=True)
x_test_o = x_test_o.astype('float64')

In [4]:
data_train_all = x_train_o.merge(y_train_o, on='Var1', how = 'left')


In [7]:
data_nolabel = data_train_all[data_train_all.Var66.isnull()]
data_label = data_train_all[data_train_all.Var66.notnull()]

In [10]:
data_nolabel.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65,Var66
1,15092.0,0.049699,0.065808,0.726800,12.94400,233.1100,0.00000,0.063192,14.19601,0.89618,...,0.059565,0.053189,0.93169,0.000000,5.0492,11.1520,24.784,14.7270,4.2204,NaN
3,14171.0,0.001417,0.708110,-0.052312,0.88978,-31.1980,0.26952,0.001407,0.41222,1.96540,...,-0.000535,0.004820,0.99930,0.745480,17.1011,7.9482,88.147,4.1408,3.4021,NaN
4,12900.0,0.020041,0.346520,0.335930,2.76130,39.0500,0.00000,0.020031,1.88591,1.29750,...,0.212410,0.030652,0.80158,0.000862,9.7670,6.7570,53.651,6.8032,2.7412,NaN
5,16499.0,0.140000,0.419930,0.290420,1.74160,7.1865,0.20062,0.177390,1.31961,1.10280,...,0.093230,0.252620,0.90677,0.051068,8.1696,5.1309,70.315,5.1909,6.3939,NaN
8,16122.0,0.094560,0.097575,0.422890,5.78220,82.3870,0.00000,0.108800,9.24851,1.27030,...,0.076875,0.104770,0.91846,0.000000,7.9041,4.5030,25.410,14.3650,2.5994,NaN


In [12]:
data_nolabel_v = data_nolabel.drop(columns=['Var1', 'Var66'])
data_nolabel_id = data_nolabel['Var1']

In [55]:
data_nolabel.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65,Var66
1,15092.0,0.049699,0.065808,0.726800,12.94400,233.1100,0.00000,0.063192,14.19601,0.89618,...,0.059565,0.053189,0.93169,0.000000,5.0492,11.1520,24.784,14.7270,4.2204,NaN
3,14171.0,0.001417,0.708110,-0.052312,0.88978,-31.1980,0.26952,0.001407,0.41222,1.96540,...,-0.000535,0.004820,0.99930,0.745480,17.1011,7.9482,88.147,4.1408,3.4021,NaN
4,12900.0,0.020041,0.346520,0.335930,2.76130,39.0500,0.00000,0.020031,1.88591,1.29750,...,0.212410,0.030652,0.80158,0.000862,9.7670,6.7570,53.651,6.8032,2.7412,NaN
5,16499.0,0.140000,0.419930,0.290420,1.74160,7.1865,0.20062,0.177390,1.31961,1.10280,...,0.093230,0.252620,0.90677,0.051068,8.1696,5.1309,70.315,5.1909,6.3939,NaN
8,16122.0,0.094560,0.097575,0.422890,5.78220,82.3870,0.00000,0.108800,9.24851,1.27030,...,0.076875,0.104770,0.91846,0.000000,7.9041,4.5030,25.410,14.3650,2.5994,NaN


In [14]:
data_nolabel_v_f = data_nolabel_v.fillna(data_nolabel_v.mean())

In [94]:
feature_columns = ['Var5', 'Var21', 'Var22', 'Var28', 'Var42', 'Var47', 'Var9', 'Var18', 'Var33', 'Var53', 'Var59', 'Var61', 'Var7', 'Var41', 'Var44', 'Var45', 'Var48', 'Var63', 'Var13', 'Var16', 'Var17', 'Var27', 'Var35', 'Var37', 'Var10', 'Var31', 'Var46', 'Var54', 'Var55']
feature_columns_all = ['Var1','Var5', 'Var21', 'Var22', 'Var28', 'Var42', 'Var47', 'Var9', 'Var18', 'Var33', 'Var53', 'Var59', 'Var61', 'Var7', 'Var41', 'Var44', 'Var45', 'Var48', 'Var63', 'Var13', 'Var16', 'Var17', 'Var27', 'Var35', 'Var37', 'Var10', 'Var31', 'Var46', 'Var54', 'Var55', 'Var66']


In [17]:
data_nolabel_selected = data_nolabel_v_f[feature_columns]

In [18]:
data_nolabel_selected.head()

,Var5,Var21,Var22,Var28,Var42,Var47,Var9,Var18,Var33,Var53,...,Var16,Var17,Var27,Var35,Var37,Var10,Var31,Var46,Var54,Var55
1,12.94400,72.291,0.59206,27.4790,0.046838,10.02700,14.19601,15.1960,26.353,0.072201,...,338.17,1.079300,0.874140,12.80700,0.89618,0.89618,-0.508100,0.279950,4.39940,4.39940
3,0.88978,21.344,0.79908,1.0323,0.351380,0.64765,0.41222,1.4122,88.100,0.241370,...,6803.70,0.053647,0.053647,2.77700,1.96540,1.96540,0.329700,0.012242,0.50528,0.88196
4,2.76130,37.371,0.79278,12.9260,0.103420,2.06480,1.88591,2.8859,68.121,0.186630,...,1046.10,0.348920,0.348920,2.94910,1.29750,1.29750,0.191060,0.150780,1.38060,1.38170
5,1.74160,44.678,1.11270,0.9835,0.066891,1.10620,1.31961,2.3813,77.544,0.212450,...,756.96,0.482190,0.393110,0.43174,2.05360,1.10280,0.190960,0.562550,1.74290,1.83190
8,5.78220,46.179,0.88756,4.2187,0.023078,3.96480,9.24851,10.2490,27.526,0.075413,...,248.82,1.466900,1.320900,12.01800,1.27030,1.27030,0.024202,0.588320,1.84670,1.84670


In [19]:
data_nolabel_selected_stand = pd.DataFrame(preprocessing.StandardScaler().fit_transform(data_nolabel_selected),columns = data_nolabel_selected.columns)

In [21]:
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.datasets.samples_generator import make_blobs

bandwidth = estimate_bandwidth(data_nolabel_selected_stand, quantile=0.2)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(data_nolabel_selected_stand)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

In [73]:
data_nolabel_id_noidx = data_nolabel_id.reset_index()['Var1']

In [74]:
data_nolabel_id_noidx.head()

0    15092.0
1    14171.0
2    12900.0
3    16499.0
4    16122.0
Name: Var1, dtype: float64

In [82]:
result = pd.DataFrame(labels, columns=["Var66"])
data_nolabel_pred = pd.concat([data_nolabel_id_noidx, data_nolabel_selected_stand, result], axis=1)

In [89]:
data_nolabel_pred.loc[data_nolabel_pred['Var66'] > 0, 'Var66'] = 1

In [99]:
data_label_fill = data_label.fillna(data_label.mean())[feature_columns_all]

,Var1,Var5,Var21,Var22,Var28,Var42,Var47,Var9,Var18,Var33,...,Var17,Var27,Var35,Var37,Var10,Var31,Var46,Var54,Var55,Var66
0,18399.0,3.63570,38.335,1.10510,0.173310,0.071334,2.53140,4.56831,6.6614,35.100,...,0.487370,0.440170,0.18022,1.61780,1.01120,0.027224,0.14443,1.50990,1.50990,0.0
2,19821.0,1.40430,65.049,3.72560,-337.820000,-0.052357,0.98259,1.54531,2.5453,131.530,...,-0.639330,-0.639330,2.77500,0.92963,0.92963,0.071085,-2.15070,1.35430,1.35430,0.0
6,17769.0,0.34907,22.961,7.61638,1.580900,0.313260,0.25256,0.38306,1.3830,216.970,...,0.097876,0.097876,1.68220,1.10920,1.10920,0.646240,0.05861,0.37046,0.37046,0.0
7,19309.0,5.58830,13.956,1.40240,0.090212,0.031600,4.49140,10.19801,11.8690,12.811,...,1.124000,1.075200,0.25703,2.44590,1.00690,-0.054839,0.26603,1.62360,1.62360,0.0
9,20728.0,1.37330,47.458,1.01400,-0.189010,0.192160,0.80964,1.28881,2.3985,83.443,...,0.366180,0.332300,-0.07318,1.69090,0.99111,0.258610,0.17642,1.08960,1.18670,0.0


In [101]:
data_tran_all  = pd.concat([data_label_fill, data_nolabel_pred], axis=0)

In [103]:
data_tran_all.head()

,Var1,Var5,Var21,Var22,Var28,Var42,Var47,Var9,Var18,Var33,...,Var17,Var27,Var35,Var37,Var10,Var31,Var46,Var54,Var55,Var66
0,18399.0,3.63570,38.335,1.10510,0.173310,0.071334,2.53140,4.56831,6.6614,35.100,...,0.487370,0.440170,0.18022,1.61780,1.01120,0.027224,0.14443,1.50990,1.50990,0.0
2,19821.0,1.40430,65.049,3.72560,-337.820000,-0.052357,0.98259,1.54531,2.5453,131.530,...,-0.639330,-0.639330,2.77500,0.92963,0.92963,0.071085,-2.15070,1.35430,1.35430,0.0
6,17769.0,0.34907,22.961,7.61638,1.580900,0.313260,0.25256,0.38306,1.3830,216.970,...,0.097876,0.097876,1.68220,1.10920,1.10920,0.646240,0.05861,0.37046,0.37046,0.0
7,19309.0,5.58830,13.956,1.40240,0.090212,0.031600,4.49140,10.19801,11.8690,12.811,...,1.124000,1.075200,0.25703,2.44590,1.00690,-0.054839,0.26603,1.62360,1.62360,0.0
9,20728.0,1.37330,47.458,1.01400,-0.189010,0.192160,0.80964,1.28881,2.3985,83.443,...,0.366180,0.332300,-0.07318,1.69090,0.99111,0.258610,0.17642,1.08960,1.18670,0.0


In [104]:
os_dfs_dict = OrderedDict()

os_dfs_dict['mean'] = pre.oversample_smote(data_tran_all, columns = data_tran_all.columns, verbose=True)

original dataset (labels): {0.0: 8606, 1.0: 397}
total: 9003
resampled dataset (labels): {0.0: 8606, 1.0: 8606}
total: 17212



In [109]:
x_test_business_id = x_test_o['Var1']
x_test = x_test_o.drop(columns=['Var1'])
x_test_f = x_test.fillna(x_test.mean())

In [111]:
# initialize decision tree classifier (sklearn)
decision_tree_classifier_sklearn = DecisionTreeClassifier(criterion="gini")

# initialize random forest classifier (sklearn)
random_forest_classifier_sklearn = RandomForestClassifier(criterion="gini", 
                                                          n_estimators = 10,
                                                          max_features = "sqrt", 
                                                          random_state = 94)

# initialize logistic regression classifier (sklearn)
log_reg_classifier_sklearn = sklm.LogisticRegression(C = 0.1, 
                                                     max_iter = 1000,
                                                     penalty="l2", 
                                                     solver="sag", 
                                                     random_state = 94)


# creating ordered dictionary for all different models (sklearn)
models_dict_sklearn = OrderedDict()
models_dict_sklearn["Decision Tree Sklearn"] = (decision_tree_classifier_sklearn, False)
models_dict_sklearn["Random Forest Sklearn"] = (random_forest_classifier_sklearn, False)
models_dict_sklearn["Logistic Regression Sklearn"] = (log_reg_classifier_sklearn, True)

# initialize dictionary to hold results for each experiment (sklearn)
experiments_dict_sklearn = OrderedDict()

In [114]:
best_period = os_dfs_dict['mean'][0]
X = best_period.iloc[:,1:-1].values
y = best_period["Var66"].values

random_forest_classifier_sklearn_roc_all = RandomForestClassifier(criterion="gini", 
                                                          n_estimators = 10,
                                                          max_features = "sqrt", 
                                                          random_state = 94)

random_forest_classifier_sklearn_roc_all.fit(X, y)
y_pred_dt = random_forest_classifier_sklearn_roc_all.predict(x_test_f[feature_columns])


In [124]:
df_y = pd.DataFrame(y_pred_dt, columns=["Is_Bankrupted"])
upload = pd.concat([x_test_business_id, df_y], axis=1)
df_y["Is_Bankrupted"].value_counts()

0    1468
1      32
Name: Is_Bankrupted, dtype: int64

In [127]:
upload = upload.astype('int32')
upload.columns=['Business_ID', 'Is_Bankrupted']
upload.to_csv('3_17.csv', index=False)